In [20]:
import pandas as pd
import string 
from math import log10
import numpy as np
df_reviews = pd.read_csv('IMDB_Dataset.csv')


df_reviews_teste = df_reviews.sample(10000)


df_filtrado_positivo = df_reviews_teste[df_reviews_teste['sentiment']=='positive']
df_filtrado_negativo = df_reviews_teste[df_reviews_teste['sentiment']=='negative']



dict_palavras_pos={}
for review in df_filtrado_positivo['review']:
    review=review.replace('<br />', " ")
    translator= str.maketrans(' ',' ', string.punctuation)
    review = review.translate(translator)
    review=set(review.lower().split())
    for palavra in review:
        if palavra in dict_palavras_pos:
            dict_palavras_pos[palavra]+=1
        else:
            dict_palavras_pos[palavra]=1


dict_palavras_neg={}
for review in df_filtrado_negativo['review']:
    review=review.replace('<br />', " ")
    translator= str.maketrans(' ',' ', string.punctuation)
    review = review.translate(translator)
    review=set(review.lower().split())
    for palavra in review:
        palavra=palavra.lower()
        if palavra in dict_palavras_neg:
            dict_palavras_neg[palavra]+=1
        else:
            dict_palavras_neg[palavra]=1


In [21]:
import copy

#calcular P(palavra|classe)

#positivo
quant_review_pos = len(df_filtrado_positivo)
dict_prob_dado_pos = {}

for palavra, quant in dict_palavras_pos.items():
    prob = quant/quant_review_pos
    dict_prob_dado_pos[palavra] = copy.deepcopy(prob)



#negativo
quant_review_neg = len(df_filtrado_negativo)
dict_prob_dado_neg = {}

for palavra, quant in dict_palavras_neg.items():
    prob = quant/quant_review_neg
    dict_prob_dado_neg[palavra] = copy.deepcopy(prob)



#calcular P(nao palavra|classe)

#positvo
dict_prob_n_palavra_dado_pos = {}
for palavra, prob in dict_prob_dado_pos.items():
    prob_n_palavra = 1 - prob
    dict_prob_n_palavra_dado_pos[palavra] = copy.deepcopy(prob_n_palavra)

#negativo
dict_prob_n_palavra_dado_neg = {}
for palavra, prob in dict_prob_dado_neg.items():
    prob_n_palavra = 1 - prob
    dict_prob_n_palavra_dado_neg[palavra] = copy.deepcopy(prob_n_palavra)

In [22]:
#Calcular P(palavra)
palavras_totais ={}

for palavra, quant in dict_palavras_pos.items():
    palavras_totais[palavra] = quant

    
for palavra, quant in dict_palavras_neg.items():
    if palavra in palavras_totais:
        palavras_totais[palavra]+=quant
    else:
        palavras_totais[palavra]=quant

prob_palavras_dict={}
for palavra in palavras_totais:
    p_palavra=palavras_totais[palavra]/len(df_reviews_teste)
    prob_palavras_dict[palavra]=p_palavra

In [23]:
#Calcular P(classe|texto)

p_negativo = 0.5
p_positivo = 0.5

# for texto in df_reviews_teste['review']:
texto=df_reviews_teste['review']
texto=texto.iloc[5]

def p_classe_dado_texto(texto, palavras_totais, dict_prob_dado_pos, dict_prob_n_palavra_dado_pos, dict_prob_n_palavra_dado_neg, prob_palavras_dict):
    p_total_positivo = 0
    p_total_negativo = 0
    epsilon = 1e-7

    for palavra in palavras_totais.keys():
        if palavra in texto:
            #positivo
            if palavra in dict_prob_dado_pos:
                p_total_positivo += np.log(dict_prob_dado_pos[palavra]+epsilon) - np.log(prob_palavras_dict[palavra]+epsilon)
            else: 
                p_total_positivo += np.log(0.0000001) - np.log(prob_palavras_dict[palavra]+epsilon)
            #negativo
            if palavra in dict_prob_dado_neg:
                p_total_negativo += np.log(dict_prob_dado_neg[palavra]+epsilon) - np.log(prob_palavras_dict[palavra]+epsilon)
            else:
                p_total_negativo += np.log(0.0000001) - np.log(prob_palavras_dict[palavra]+epsilon)

        else:
            #positivo
            if palavra in dict_prob_n_palavra_dado_pos:
                p_total_positivo += np.log(dict_prob_n_palavra_dado_pos[palavra]+epsilon) - np.log(1-prob_palavras_dict[palavra]+epsilon)
            else: 
                p_total_positivo += np.log(0.9999) - np.log(prob_palavras_dict[palavra]+epsilon)
            #negativo
            if palavra in dict_prob_n_palavra_dado_neg:
                p_total_negativo += np.log(dict_prob_n_palavra_dado_neg[palavra]+epsilon) - np.log(1-prob_palavras_dict[palavra]+epsilon)
            else:  
                p_total_negativo += np.log(0.9999) - np.log(prob_palavras_dict[palavra]+epsilon)
        
    p_total_positivo+=np.log(p_positivo)
    p_total_negativo += np.log(p_negativo)
    return p_total_positivo, p_total_negativo

print(p_classe_dado_texto("Love love love".split(), palavras_totais, dict_prob_dado_pos, dict_prob_n_palavra_dado_pos, dict_prob_n_palavra_dado_neg, prob_palavras_dict))

(187178.9141897779, 214885.58711293474)
